In [33]:
import os
os.getcwd()

'/media/notebooks/SP18-1-RYDAJO/Assignments/HW5'

In [35]:
cd /media/notebooks/SP18-1-RYDAJO/w209

/media/notebooks/SP18-1-RYDAJO/w209


In [39]:
!pwd

/media/notebooks/SP18-1-RYDAJO/w209


In [37]:
# Initial setup. Just run this cell once.
import numpy as np
import matplotlib.pyplot as plt
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

__Paths to Main data in HDFS on Altiscale__

### `buildStripes.py`

In [156]:
%%writefile chord_counter.py
##!~/anaconda2/bin/python
# -*- coding: utf-8 -*-

from __future__ import division
import re
import mrjob
import json, ast
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep
import collections as cl
import itertools

class MRchordcounter(MRJob):
    
    OUTPUT_PROTOCOL = RawProtocol
    
    def steps(self):
        return [MRStep(
                mapper = self.mapper,
                reducer = self.reducer
                )]
      
        
    def mapper(self, _, line):
        line = line.strip()
        data = re.split(";",line.lower())
        song = data[0]
        genre = data[1]
        songKey = data[2]
        chordString = re.sub(r'[\[\]]','',data[3])
        chords = re.split(",",chordString)
        for chord in chords:
            yield chord, 1
            
    def reducer(self, key, value):
        count = sum(value)
        yield key, str(count)
        
if __name__ == '__main__':
    MRchordcounter.run()

Overwriting chord_counter.py


In [157]:
!python chord_counter.py processed_songs_v1.txt > chord_count_list.txt

No configs found; falling back on auto-configuration
Creating temp directory /tmp/chord_counter.root.20181203.190139.188833
Running step 1 of 1...
Streaming final output from /tmp/chord_counter.root.20181203.190139.188833/output...
Removing temp directory /tmp/chord_counter.root.20181203.190139.188833...


In [159]:
%%writefile chord_counter_detailed_OG.py
##!~/anaconda2/bin/python
# -*- coding: utf-8 -*-

from __future__ import division
import re
import mrjob
import json, ast
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep
import collections as cl
import itertools

class MRchordCounter_detailed(MRJob):
    
    OUTPUT_PROTOCOL = RawProtocol
    
    def steps(self):
        return [MRStep(
                mapper_init = self.mapper_init,
                mapper = self.mapper,
                mapper_final = self.mapper_final,
                reducer = self.reducer
                )]
    
    def mapper_init(self):
        self.chordProgressions = {}        
        
    def mapper(self, _, line):
        line = line.strip()
        data = re.split(";",line.lower())
        song = data[0]
        genre = data[1]
        songKey = data[2]
        chordString = re.sub(r'[\[\]]','',data[3])
        chords = re.split(",",chordString)
        # iterate through all two-chord progressionss in order
        for chord in chords:
            # see if the songKey the dictionary for this firstSong
            if songKey not in self.chordProgressions.keys():
                # create sub-array for data within this songKey
                self.chordProgressions[songKey] = {}
                self.chordProgressions[songKey][genre] = {}
                self.chordProgressions[songKey][genre][secondChord] = 1
                self.chordProgressions[songKey]["all_Genres"] = {}
                self.chordProgressions[songKey]["all_Genres"][secondChord] = 1
                # see if the chord is found in the all genres dictionary for all songKeys
                if secondChord not in self.chordProgressions["all_Keys"]["all_Genres"]:    
                    self.chordProgressions["all_Keys"]["all_Genres"][secondChord] = 1
                else:
                    self.chordProgressions["all_Keys"]["all_Genres"][secondChord] += 1
                # see if the current genre is found in the all songKeys dictionary
                if genre not in self.chordProgressions["all_Keys"].keys():
                    self.chordProgressions["all_Keys"][genre] = {}
                    self.chordProgressions["all_Keys"][genre][secondChord] = 1
                else:
                    # check to see if there is an existing entry to add to
                    if secondChord not in self.chordProgressions["all_Keys"][genre]:
                        self.chordProgressions["all_Keys"][genre][secondChord] = 1
                    else:
                        self.chordProgressions["all_Keys"][genre][secondChord] += 1
            # see if the genre is in the deictionary for this songKey
            elif genre not in self.chordProgressions[songKey].keys():
                self.chordProgressions[songKey][genre] = {}
                self.chordProgressions[songKey][genre][secondChord] = 1
                # see if the chord is found in the all genres dictionary for all songKeys
                if secondChord not in self.chordProgressions["all_Keys"]["all_Genres"]:    
                    self.chordProgressions["all_Keys"]["all_Genres"][secondChord] = 1
                else:
                    self.chordProgressions["all_Keys"]["all_Genres"][secondChord] += 1
                # see if the current genre is found in the all songKeys dictionary
                if genre not in self.chordProgressions["all_Keys"].keys():
                    self.chordProgressions["all_Keys"][genre] = {}
                    self.chordProgressions["all_Keys"][genre][secondChord] = 1
                else:
                    # check to see if there is an existing entry to add to
                    if secondChord not in self.chordProgressions["all_Keys"][genre]:
                        self.chordProgressions["all_Keys"][genre][secondChord] = 1
                    else:
                        self.chordProgressions["all_Keys"][genre][secondChord] += 1
                # see if the second chord is in the data for all genres in this songKey
                if secondChord not in self.chordProgressions[songKey]["all_Genres"]:
                    self.chordProgressions[songKey]["all_Genres"][secondChord] = 1
                else:
                    self.chordProgressions[songKey]["all_Genres"][secondChord] += 1
            # otherwise we just need to increase the counts appropriately
            elif secondChord not in self.chordProgressions[songKey][genre]:
                self.chordProgressions[songKey][genre][secondChord] = 1
                # see if the chord is found in the all genres dictionary for all songKeys
                if secondChord not in self.chordProgressions["all_Keys"]["all_Genres"]:    
                    self.chordProgressions["all_Keys"]["all_Genres"][secondChord] = 1
                else:
                    self.chordProgressions["all_Keys"]["all_Genres"][secondChord] += 1
                # we know the genre is in "all_keys" dictionary from the "elif genre" above
                # but we still need to see if the second chord is in the data for this genre in all keys
                if secondChord not in self.chordProgressions["all_Keys"][genre]:
                    self.chordProgressions["all_Keys"][genre][secondChord] = 1
                else:
                    self.chordProgressions["all_Keys"][genre][secondChord] += 1
                # we know the songKey is in "firstChord" dictionary from the "elif songKey" above
                # but we still need to see if the second chord is in the data for all genres in this songKey
                if secondChord not in self.chordProgressions[songKey]["all_Genres"]:
                    self.chordProgressions[songKey]["all_Genres"][secondChord] = 1
                else:
                    self.chordProgressions[songKey]["all_Genres"][secondChord] += 1    
            else:
                self.chordProgressions[songKey][genre][secondChord] += 1
                self.chordProgressions[songKey]["all_Genres"][secondChord] += 1
                self.chordProgressions["all_Keys"]["all_Genres"][secondChord] += 1
                self.chordProgressions["all_Keys"][genre][secondChord] += 1
                
    def mapper_final(self):
        for key in self.chordProgressions.keys():
            # print key+";"+json.dumps(self.chordProgressions[key])
            yield key, json.dumps(self.chordProgressions[key])

    def reducer(self, key, value):
        songKey= key
        self.chordProgressions = {}
        self.chordProgressions[songKey] = {}
        for v in value:
            chord_dict = json.loads(v)
            for genre in chord_dict.keys():
                if genre not in self.chordProgressions[songKey].keys():
                    # create sub-array for data within this songKey
                    self.chordProgressions[songKey][genre] = {}
                    for secondChord in chord_dict[songKey][genre]:
                        if secondChord not in self.chordProgressions[songKey][genre]:
                            self.cxhordProgressions[songKey][genre][secondChord] = 1
                        else:
                            self.chordProgressions[songKey][genre][secondChord] += 1
        yield songKey, json.dumps(self.chordProgressions[songKey]) 

        
if __name__ == '__main__':
    MRchordCounter_detailed.run()

Overwriting chord_counter_detailed.py


In [342]:
%%writefile chord_counter_detailed.py
##!~/anaconda2/bin/python
# -*- coding: utf-8 -*-

from __future__ import division
import re
import mrjob
import json, ast
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep
import collections as cl
import itertools

class MRchordCounter_detailed(MRJob):
    
    OUTPUT_PROTOCOL = RawProtocol
    
    def steps(self):
        return [MRStep(
                mapper_init = self.mapper_init,
                mapper = self.mapper,
                mapper_final = self.mapper_final,
                reducer = self.reducer
                )]
    
    def mapper_init(self):
        outputChordKeys = ["G-Major", "C-Major", "D-Major", "E-Minor", "A-Minor", "A-Major", "F-Major", "E-Major", "B-Minor",  "B-Major", "C-Add-9", "D-Minor", "B-Flat-Major", "A-Flat-Major", "D-Flat-5", "A-Suspended-2", "A-Minor-7", "E-Flat-Major", "B-Flat-Five", "A-Flat-5", "A-Flat-Minor", "B-Flat-Suspended-2", "F-Minor", "E-Minor-7", "F-Minor-7", "D-suspended-4", "D-7", "F-Suspended-2", "D-Flat-Major", "E-Major-9", "F-5", "E-Flat-5", "D-Suspended-2", "E-Flat-Minor-7", "C-Major-7", "G-Flat-Minor", "C-Suspended-2", "F-Major-7", "A-Minor-9", "B-7", "C-5", "C-Minor", "B-Minor-7", "C-Minor-7", "G-6", "A-Major-9", "E-7", "G-Minor", "G-Major-7"]
        #outputChordKeys = ["G-Major", "C-Major", "D-Major", "E-Minor", "A-Minor", "A-Major", "F-Major", "E-Major", "B-Minor",  "B-Major", "C-Add-9", "D-Minor", "B-Flat-Major", "A-Flat-Major", "D-Flat-5", "A-Suspended-2", "A-Minor-7", "E-Flat-Major", "B-Flat-Five", "A-Flat-5", "A-Flat-Minor", "B-Flat-Suspended-2", "F-Minor", "E-Minor-7", "F-Minor-7", "D-suspended-4", "D-7", "F-Suspended-2", "D-Flat-Major", "E-Major-9", "F-5", "E-Flat-5", "D-Suspended-2", "E-Flat-Minor-7", "C-Major-7", "G-Flat-Minor", "C-Suspended-2", "F-Major-7", "A-Minor-9", "B-7", "C-5", "C-Minor", "B-Minor-7", "C-Minor-7", "G-6", "A-Major-9", "E-7", "G-Minor", "G-Major-7"]
        genres = ["all_Genres", "rock", "pop", "folk", "country"]
        songKeys= ["all_Keys", "a-flat-major", "a-major", "a-minor", "b-flat-major", "b-major", "b-minor", "c-major", "d-flat-major", "d-flat-minor", "d-major", "d-minor", "e-flat-major", "e-flat-minor", "e-major", "e-minor", "f-major", "f-minor", "f-sharp-major", "g-major", "g-minor"]
        # create master dictionary with fixed indexes
        self.chordProgressions = {}
        for songKey in songKeys:
            self.chordProgressions[songKey] = {}
            for genre in genres:
                self.chordProgressions[songKey][genre] = {}
                # set default values to 0 for eached chord in fixed index
                for chord in outputChordKeys:
                    self.chordProgressions[songKey][genre][chord] = 0
        self.chordConversion = {"g": "G-Major", "c": "C-Major", "d": "D-Major", "em": "E-Minor", "am": "A-Minor", "a": "A-Major", "f": "F-Major", "e": "E-Major", "bm": "B-Minor", "b": "B-Major", "cadd9": "C-Add-9", "dm": "D-Minor", "bb": "B-Flat-Major", "ab": "A-Flat-Major", "db5": "D-Flat-5", "asus2": "A-Suspended-2", "am7": "A-Minor-7", "eb": "E-Flat-Major", "bb5": "B-Flat-Five", "ab5": "A-Flat-5", "abm": "A-Flat-Minor", "bbsus2": "B-Flat-Suspended-2", "fm": "F-Minor", "em7": "E-Minor-7", "fm7": "F-Minor-7", "dsus4": "D-suspended-4", "d7": "D-7", "fsus2": "F-Suspended-2", "db": "D-Flat-Major", "emaj9": "E-Major-9", "f5": "F-5", "eb5": "E-Flat-5", "dsus2": "D-Suspended-2", "ebm7": "E-Flat-Minor-7", "cmaj7": "C-Major-7", "gbm": "G-Flat-Minor", "csus2": "C-Suspended-2", "fmaj7": "F-Major-7", "am9": "A-Minor-9", "b7": "B-7", "c5": "C-5", "cm": "C-Minor", "bm7": "B-Minor-7", "c7m": "C-Minor-7", "g6": "G-6", "amaj9": "A-Major-9", "e7": "E-7", "gm": "G-Minor", "gmaj7": "G-Major-7"}           
            
        
    def mapper(self, _, line):
        line = line.strip()
        data = re.split(";",line.lower())
        song = data[0]
        genre = data[1]
        songKey = data[2].replace(" ","-")
        chordString = re.sub(r'[\[\]]','',data[3])
        chords = re.split(",",chordString)
        # dictionary and array of values for processing
        chordsWeCareAbout = ["a", "am", "b", "bb", "bm", "c", "cadd9", "d", "db", "dm", "e", "em", "fm", "g", "gm"]
        # iterate through all two-chord progressionss in order
        for chord in chords:
            if chord.replace("'","") in chordsWeCareAbout:
                chord = self.chordConversion[chord.replace("'","")]
                # see if the songKey the dictionary for this firstSong
                if songKey not in self.chordProgressions.keys():
                    # create sub-array for data within this songKey
                    self.chordProgressions[songKey] = {}
                    self.chordProgressions[songKey][genre] = {}
                    self.chordProgressions[songKey][genre][chord] = 1
                    self.chordProgressions[songKey]["all_Genres"] = {}
                    self.chordProgressions[songKey]["all_Genres"][chord] = 1
                    # see if the chord is found in the all genres dictionary for all songKeys
                    if chord not in self.chordProgressions["all_Keys"]["all_Genres"]:    
                        self.chordProgressions["all_Keys"]["all_Genres"][chord] = 1
                    else:
                        self.chordProgressions["all_Keys"]["all_Genres"][chord] += 1
                    # see if the current genre is found in the all songKeys dictionary
                    if genre not in self.chordProgressions["all_Keys"].keys():
                        self.chordProgressions["all_Keys"][genre] = {}
                        self.chordProgressions["all_Keys"][genre][chord] = 1
                    else:
                        # check to see if there is an existing entry to add to
                        if chord not in self.chordProgressions["all_Keys"][genre]:
                            self.chordProgressions["all_Keys"][genre][chord] = 1
                        else:
                            self.chordProgressions["all_Keys"][genre][chord] += 1
                # see if the genre is in the deictionary for this songKey
                elif genre not in self.chordProgressions[songKey].keys():
                    self.chordProgressions[songKey][genre] = {}
                    self.chordProgressions[songKey][genre][chord] = 1
                    # see if the chord is found in the all genres dictionary for all songKeys
                    if chord not in self.chordProgressions["all_Keys"]["all_Genres"]:    
                        self.chordProgressions["all_Keys"]["all_Genres"][chord] = 1
                    else:
                        self.chordProgressions["all_Keys"]["all_Genres"][chord] += 1
                    # see if the current genre is found in the all songKeys dictionary
                    if genre not in self.chordProgressions["all_Keys"].keys():
                        self.chordProgressions["all_Keys"][genre] = {}
                        self.chordProgressions["all_Keys"][genre][chord] = 1
                    else:
                        # check to see if there is an existing entry to add to
                        if chord not in self.chordProgressions["all_Keys"][genre]:
                            self.chordProgressions["all_Keys"][genre][chord] = 1
                        else:
                            self.chordProgressions["all_Keys"][genre][chord] += 1
                    # see if the second chord is in the data for all genres in this songKey
                    if chord not in self.chordProgressions[songKey]["all_Genres"]:
                        self.chordProgressions[songKey]["all_Genres"][chord] = 1
                    else:
                        self.chordProgressions[songKey]["all_Genres"][chord] += 1
                # otherwise we just need to increase the counts appropriately
                elif chord not in self.chordProgressions[songKey][genre]:
                    self.chordProgressions[songKey][genre][chord] = 1
                    # see if the chord is found in the all genres dictionary for all songKeys
                    if chord not in self.chordProgressions["all_Keys"]["all_Genres"]:    
                        self.chordProgressions["all_Keys"]["all_Genres"][chord] = 1
                    else:
                        self.chordProgressions["all_Keys"]["all_Genres"][chord] += 1
                    # we know the genre is in "all_keys" dictionary from the "elif genre" above
                    # but we still need to see if the second chord is in the data for this genre in all keys
                    if chord not in self.chordProgressions["all_Keys"][genre]:
                        self.chordProgressions["all_Keys"][genre][chord] = 1
                    else:
                        self.chordProgressions["all_Keys"][genre][chord] += 1
                    # we know the songKey is in "firstChord" dictionary from the "elif songKey" above
                    # but we still need to see if the second chord is in the data for all genres in this songKey
                    if chord not in self.chordProgressions[songKey]["all_Genres"]:
                        self.chordProgressions[songKey]["all_Genres"][chord] = 1
                    else:
                        self.chordProgressions[songKey]["all_Genres"][chord] += 1    
                else:
                    self.chordProgressions[songKey][genre][chord] += 1
                    self.chordProgressions[songKey]["all_Genres"][chord] += 1
                    self.chordProgressions["all_Keys"]["all_Genres"][chord] += 1
                    self.chordProgressions["all_Keys"][genre][chord] += 1
                
    def mapper_final(self):
        for key in self.chordProgressions.keys():
            # print key+";"+json.dumps(self.chordProgressions[key])
            yield key, json.dumps(self.chordProgressions[key])

    def reducer(self, key, value):
        self.chordProgressions = {}
        for v in value:
            chord_dict = json.loads(v)
            for genre in chord_dict.keys():
                if genre not in self.chordProgressions.keys():
                    # create sub-array for data within this songKey
                    self.chordProgressions[genre] = {}
                    for chord in chord_dict[genre]:
                        if chord not in self.chordProgressions[genre]:
                            self.chordProgressions[genre][chord] = chord_dict[genre][chord]
                        else:
                            self.chordProgressions[genre][chord] += chord_dict[genre][chord]
        yield key, json.dumps(self.chordProgressions) 
        
if __name__ == '__main__':
    MRchordCounter_detailed.run()

Overwriting chord_counter_detailed.py


In [343]:
!python chord_counter_detailed.py processed_songs_v1.txt > chord_count_detailed_list_fixed.txt

No configs found; falling back on auto-configuration
Creating temp directory /tmp/chord_counter_detailed.root.20181208.235131.754588
Running step 1 of 1...
Streaming final output from /tmp/chord_counter_detailed.root.20181208.235131.754588/output...
Removing temp directory /tmp/chord_counter_detailed.root.20181208.235131.754588...


In [290]:
%%writefile normalizedChordCounts_detailed.py
#!~/anaconda2/bin/python
# -*- coding: utf-8 -*-

from __future__ import division
import collections
import re
import json
import math
import numpy as np
import itertools
import mrjob
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep
from collections import defaultdict
import collections as cl
import itertools

class MRnormalizedChordCounts_detailed(MRJob):
    
  #START SUDENT CODE531_INV_INDEX
    SORT_VALUES = True
    def steps(self):
        JOBCONF_STEP = { 
          ######### IMPORTANT: THIS WILL HAVE NO EFFECT IN -r local MODE. MUST USE -r hadoop FOR SORTING #############
            'mapreduce.job.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapreduce.partition.keycomparator.options':'-k1',
        }
        return [
                MRStep(
                    jobconf=JOBCONF_STEP,
                    mapper_init = self.mapper_init,
                    mapper = self.mapper,
                    mapper_final = self.mapper_final,
                    reducer_init =self.reducer_init,
                    reducer = self.reducer,
                    reducer_final = self.reducer_final)
                ]
    
    def mapper_init(self):
        self.chordProgressions = {}
    
    def mapper(self, _, line):
        key, value = line.strip().split('\t')
        self.chordProgressions[key] = {}
        self.chordProgressions[key] = json.loads(value)
        # find the total count of chord progressions in the diectionary
        for genre in self.chordProgressions[key].keys():
            normalizationCount = 0
            for nextChord in self.chordProgressions[key][genre].keys():
                normalizationCount += int(self.chordProgressions[key][genre][nextChord])
            for nextChord in self.chordProgressions[key][genre].keys():
                if normalizationCount != 0:
                    self.chordProgressions[key][genre][nextChord] = int(self.chordProgressions[key][genre][nextChord])/normalizationCount
                else:
                    self.chordProgressions[key][genre][nextChord]

    def mapper_final(self):
        for key in self.chordProgressions.keys():
            yield key , json.dumps(self.chordProgressions[key])
    
    
    def reducer_init(self):
        self.full_dict = {}
    
    # the reducer is purely being used for sortation
    def reducer(self, key, value):
        songKey = key
        #songKey = str('"'+key+'"')
        # songKey  = key.replace(r"'","")
        # print str('"'+songKey+'", ')
        for v in value:
            temp = json.loads(v)
        self.full_dict[songKey] = {}
        self.full_dict[songKey] = temp
        # print songKey, json.dumps(temp), "\n"
        
    def reducer_final(self):
        #yield "chord_dict", json.dumps(self.full_dict)
        print json.dumps(self.full_dict), "\n"
        
if __name__ == '__main__':
    MRnormalizedChordCounts_detailed.run()

Overwriting normalizedChordCounts_detailed.py


In [344]:
!python normalizedChordCounts_detailed.py chord_count_detailed_list_fixed.txt > normalized_chord_count_detailed_list_fixed.txt

No configs found; falling back on auto-configuration
Creating temp directory /tmp/normalizedChordCounts_detailed.root.20181208.235137.446666
Running step 1 of 1...
Streaming final output from /tmp/normalizedChordCounts_detailed.root.20181208.235137.446666/output...
Removing temp directory /tmp/normalizedChordCounts_detailed.root.20181208.235137.446666...


In [337]:
%%writefile buildStripes_detailed_OG.py
##!~/anaconda2/bin/python
# -*- coding: utf-8 -*-

from __future__ import division
import re
import mrjob
import json, ast
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep
import collections as cl
import itertools

class MRbuildStripes_detailed(MRJob):
    
    OUTPUT_PROTOCOL = RawProtocol
    
    def steps(self):
        return [MRStep(
                mapper_init = self.mapper_init,
                mapper = self.mapper,
                mapper_final = self.mapper_final,
                reducer = self.reducer
                )]
    
    def mapper_init(self):
        self.chordProgressions = {}
        firstChords = ["a", "a5", "a6", "a7sus4", "aadd9", "ab", "ab5", "abm", "abm7", "absus4", "am", "am7", "am9", "amaj9", "ao", "asus", "asus2", "asus4", "b", "b7", "bb", "bb5", "bb6sus2", "bb7", "bbm", "bbm7", "bbsus2", "bbsus4", "bm", "bm7", "bsus4", "c", "c5", "c7", "c7m", "cadd9", "cm", "cmaj7", "csus2", "d", "d5", "d7", "dadd4", "db", "db5", "dm", "dm7", "dsus", "dsus2", "dsus4", "e", "e5", "e7", "eadd9", "eb", "eb5", "ebm", "ebm7", "ebsus2", "em", "em7", "emaj9", "f", "f5", "f7", "fm", "fm7", "fmaj7", "fsus2", "g", "g13", "g5", "g6", "g7", "g9", "gb", "gb5", "gbm", "gbm7", "gbm9", "gm", "gmaj7", "go", "gsus", "gsus4"]
        outputChordKeys = ["G-Major", "C-Major", "D-Major", "E-Minor", "A-Minor", "A-Major", "F-Major", "E-Major", "B-Minor",  "B-Major", "C-Add-9", "D-Minor", "B-Flat-Major", "A-Flat-Major", "D-Flat-5", "A-Suspended-2", "A-Minor-7", "E-Flat-Major", "B-Flat-Five", "A-Flat-5", "A-Flat-Minor", "B-Flat-Suspended-2", "F-Minor", "E-Minor-7", "F-Minor-7", "D-suspended-4", "D-7", "F-Suspended-2", "D-Flat-Major", "E-Major-9", "F-5", "E-Flat-5", "D-Suspended-2", "E-Flat-Minor-7", "C-Major-7", "G-Flat-Minor", "C-Suspended-2", "F-Major-7", "A-Minor-9", "B-7", "C-5", "C-Minor", "B-Minor-7", "C-Minor-7", "G-6", "A-Major-9", "E-7", "G-Minor", "G-Major-7"]
        genres = ["all_Genres", "rock", "pop", "folk", "country"]
        songKeys= ["all_Keys", "a flat major", "a major", "a minor", "b flat major", "b major", "b minor", "c major", "d flat major", "d flat minor", "d major", "d minor", "e flat major", "e flat minor", "e major", "e minor", "f major", "f minor", "f sharp major", "g major", "g minor"]
        # create master dictionary with fixed indexes
        self.chordProgressions = {}
        for firstChord in firstChords:
            self.chordProgressions[firstChord] = {}
            for songKey in songKeys:
                self.chordProgressions[firstChord][songKey] = {}
                for genre in genres:
                    self.chordProgressions[firstChord][songKey][genre] = {}
                    # set default values to 0 for eached chord in fixed index
                    for chord in outputChordKeys:
                        self.chordProgressions[firstChord][songKey][genre][chord] = 0
        self.chordConversion = {"a5": "A-5", "a6": "A-6", "a7sus4": "A-Suspended-4", "aadd9": "A-Add-9", "abm7": "A-Flat-Minor-7", "absus4": "A-Flat-Suspended-4", "ao": "A-Diminished", "asus": "A-Suspended", "asus4": "A-Suspended-4", "bb6sus2": "B-Flat-6-Suspended-2", "bb7": "B-Flat-7", "bbm": "B-Flat-Minor", "bbm7": "B-Flat-Minor-7", "bbsus4": "B-Flat-Suspended-4", "bsus4": "B-Suspended-4", "c7": "C-7", "d5": "D-5", "dadd4": "D-Add-4", "dm7": "D-Minor-7", "dsus": "D-Suspended", "e5": "E-5", "eadd9": "E-Add-9", "ebm": "E-Flat-Minor", "ebsus2": "E-Flat-Suspended-2", "f7": "F-7", "g13": "G-13", "g5": "G-5", "g7": "G-7", "g9": "G-9", "gb": "G-Flat-Major", "gb5": "G-Flat-5", "gbm7": "G-Flat-Minor-7", "gbm9": "G-Flat-Minor-9", "go": "G-Diminished", "gsus": "G-Suspended", "gsus4": "G-Suspended-4", "g": "G-Major", "c": "C-Major", "d": "D-Major", "em": "E-Minor", "am": "A-Minor", "a": "A-Major", "f": "F-Major", "e": "E-Major", "bm": "B-Minor",  "b": "B-Major", "cadd9": "C-Add-9", "dm": "D-Minor", "bb": "B-Flat-Major", "ab": "A-Flat-Major", "db5": "D-Flat-5", "asus2": "A-Suspended-2", "am7": "A-Minor-7", "eb": "E-Flat-Major", "bb5": "B-Flat-Five", "ab5": "A-Flat-5", "abm": "A-Flat-Minor", "bbsus2": "B-Flat-Suspended-2", "fm": "F-Minor", "em7": "E-Minor-7", "fm7": "F-Minor-7", "dsus4": "D-suspended-4", "d7": "D-7", "fsus2": "F-Suspended-2", "db": "D-Flat-Major", "emaj9": "E-Major-9", "f5": "F-5", "eb5": "E-Flat-5", "dsus2": "D-Suspended-2", "ebm7": "E-Flat-Minor-7", "cmaj7": "C-Major-7", "gbm": "G-Flat-Minor", "csus2": "C-Suspended-2", "fmaj7": "F-Major-7", "am9": "A-Minor-9", "b7": "B-7", "c5": "C-5", "cm": "C-Minor", "bm7": "B-Minor-7", "c7m": "C-Minor-7", "g6": "G-6", "amaj9": "A-Major-9", "e7": "E-7", "gm": "G-Minor", "gmaj7": "G-Major-7"}
        
    def mapper(self, _, line):
        line = line.strip()
        data = re.split(";",line.lower())
        song = data[0]
        genre = data[1]
        songKey = data[2].replace(" ","-")
        chordString = re.sub(r'[\[\]]','',data[3])
        chords = re.split(",",chordString)
        # iterate through all two-chord progressionss in order
        for i in range(0, len(chords)-1):
            # create a dictionary key that includes the "key" (tone) of the song
            firstChord = self.chordConversion[chords[i].replace("'","")]
            secondChord = self.chordConversion[chords[i+1].replace("'","")]
            if firstChord == secondChord:
                # we want to ignore repeated chords
                doNothing = 0
            # if this key-chord combination is not an existing key in the dictionary
            elif firstChord not in self.chordProgressions.keys():
                self.chordProgressions[firstChord] = {}
                # create sub-arrays for data within this songKey
                self.chordProgressions[firstChord][songKey] = {}
                self.chordProgressions[firstChord][songKey][genre] = {}
                self.chordProgressions[firstChord][songKey][genre][secondChord] = 1
                # include a way to store data across all genres
                self.chordProgressions[firstChord][songKey]["all_Genres"] = {}
                self.chordProgressions[firstChord][songKey]["all_Genres"][secondChord] = 1
                # add this data to the master dictionary for all songKeys and all genres
                self.chordProgressions[firstChord]["all_Keys"] = {}
                self.chordProgressions[firstChord]["all_Keys"]["all_Genres"] = {}
                self.chordProgressions[firstChord]["all_Keys"]["all_Genres"][secondChord] = 1
                # create a way to store data for specific genres for all songKeys
                self.chordProgressions[firstChord]["all_Keys"][genre] = {}
                self.chordProgressions[firstChord]["all_Keys"][genre][secondChord] = 1
            # see if the songKey the dictionary for this firstSong
            elif songKey not in self.chordProgressions[firstChord].keys():
                # create sub-array for data within this songKey
                self.chordProgressions[firstChord][songKey] = {}
                self.chordProgressions[firstChord][songKey][genre] = {}
                self.chordProgressions[firstChord][songKey][genre][secondChord] = 1
                self.chordProgressions[firstChord][songKey]["all_Genres"] = {}
                self.chordProgressions[firstChord][songKey]["all_Genres"][secondChord] = 1
                # see if the chord is found in the all genres dictionary for all songKeys
                if secondChord not in self.chordProgressions[firstChord]["all_Keys"]["all_Genres"]:    
                    self.chordProgressions[firstChord]["all_Keys"]["all_Genres"][secondChord] = 1
                else:
                    self.chordProgressions[firstChord]["all_Keys"]["all_Genres"][secondChord] += 1
                # see if the current genre is found in the all songKeys dictionary
                if genre not in self.chordProgressions[firstChord]["all_Keys"].keys():
                    self.chordProgressions[firstChord]["all_Keys"][genre] = {}
                    self.chordProgressions[firstChord]["all_Keys"][genre][secondChord] = 1
                else:
                    # check to see if there is an existing entry to add to
                    if secondChord not in self.chordProgressions[firstChord]["all_Keys"][genre]:
                        self.chordProgressions[firstChord]["all_Keys"][genre][secondChord] = 1
                    else:
                        self.chordProgressions[firstChord]["all_Keys"][genre][secondChord] += 1
            # see if the genre is in the deictionary for this songKey
            elif genre not in self.chordProgressions[firstChord][songKey].keys():
                self.chordProgressions[firstChord][songKey][genre] = {}
                self.chordProgressions[firstChord][songKey][genre][secondChord] = 1
                # see if the chord is found in the all genres dictionary for all songKeys
                if secondChord not in self.chordProgressions[firstChord]["all_Keys"]["all_Genres"]:    
                    self.chordProgressions[firstChord]["all_Keys"]["all_Genres"][secondChord] = 1
                else:
                    self.chordProgressions[firstChord]["all_Keys"]["all_Genres"][secondChord] += 1
                # see if the current genre is found in the all songKeys dictionary
                if genre not in self.chordProgressions[firstChord]["all_Keys"].keys():
                    self.chordProgressions[firstChord]["all_Keys"][genre] = {}
                    self.chordProgressions[firstChord]["all_Keys"][genre][secondChord] = 1
                else:
                    # check to see if there is an existing entry to add to
                    if secondChord not in self.chordProgressions[firstChord]["all_Keys"][genre]:
                        self.chordProgressions[firstChord]["all_Keys"][genre][secondChord] = 1
                    else:
                        self.chordProgressions[firstChord]["all_Keys"][genre][secondChord] += 1
                # see if the second chord is in the data for all genres in this songKey
                if secondChord not in self.chordProgressions[firstChord][songKey]["all_Genres"]:
                    self.chordProgressions[firstChord][songKey]["all_Genres"][secondChord] = 1
                else:
                    self.chordProgressions[firstChord][songKey]["all_Genres"][secondChord] += 1
            # otherwise we just need to increase the counts appropriately
            elif secondChord not in self.chordProgressions[firstChord][songKey][genre]:
                self.chordProgressions[firstChord][songKey][genre][secondChord] = 1
                # see if the chord is found in the all genres dictionary for all songKeys
                if secondChord not in self.chordProgressions[firstChord]["all_Keys"]["all_Genres"]:    
                    self.chordProgressions[firstChord]["all_Keys"]["all_Genres"][secondChord] = 1
                else:
                    self.chordProgressions[firstChord]["all_Keys"]["all_Genres"][secondChord] += 1
                # we know the genre is in "all_keys" dictionary from the "elif genre" above
                # but we still need to see if the second chord is in the data for this genre in all keys
                if secondChord not in self.chordProgressions[firstChord]["all_Keys"][genre]:
                    self.chordProgressions[firstChord]["all_Keys"][genre][secondChord] = 1
                else:
                    self.chordProgressions[firstChord]["all_Keys"][genre][secondChord] += 1
                # we know the songKey is in "firstChord" dictionary from the "elif songKey" above
                # but we still need to see if the second chord is in the data for all genres in this songKey
                if secondChord not in self.chordProgressions[firstChord][songKey]["all_Genres"]:
                    self.chordProgressions[firstChord][songKey]["all_Genres"][secondChord] = 1
                else:
                    self.chordProgressions[firstChord][songKey]["all_Genres"][secondChord] += 1    
            else:
                self.chordProgressions[firstChord][songKey][genre][secondChord] += 1
                self.chordProgressions[firstChord][songKey]["all_Genres"][secondChord] += 1
                self.chordProgressions[firstChord]["all_Keys"]["all_Genres"][secondChord] += 1
                self.chordProgressions[firstChord]["all_Keys"][genre][secondChord] += 1
                
                
    def mapper_final(self):
        for key in self.chordProgressions.keys():
            # print key+";"+json.dumps(self.chordProgressions[key])
            yield key, json.dumps(self.chordProgressions[key])

    def reducer(self, key, value):
        firstChord = key
        self.chordProgressions = {}
        self.chordProgressions[firstChord] = {}
        for v in value:
            chord_dict = json.loads(v)
            for songKey in chord_dict.keys():
                if songKey not in self.chordProgressions[firstChord].keys():
                    # create sub-array for data within this songKey
                    self.chordProgressions[firstChord][songKey] = {}
                for genre in chord_dict[songKey].keys():
                    if genre not in self.chordProgressions[firstChord][songKey].keys():
                        self.chordProgressions[firstChord][songKey][genre] = {}
                    for secondChord in chord_dict[songKey][genre]:
                        if secondChord not in self.chordProgressions[firstChord][songKey][genre]:
                            self.chordProgressions[firstChord][songKey][genre][secondChord] = chord_dict[songKey][genre][secondChord]
                        else:
                            self.chordProgressions[firstChord][songKey][genre][secondChord] += chord_dict[songKey][genre][secondChord]
        yield firstChord, json.dumps(self.chordProgressions[firstChord]) 

        
if __name__ == '__main__':
    MRbuildStripes_detailed.run()

Overwriting buildStripes_detailed.py


In [345]:
%%writefile buildStripes_detailed.py
##!~/anaconda2/bin/python
# -*- coding: utf-8 -*-

from __future__ import division
import re
import mrjob
import json, ast
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep
import collections as cl
import itertools

class MRbuildStripes_detailed(MRJob):
    
    OUTPUT_PROTOCOL = RawProtocol
    
    def steps(self):
        return [MRStep(
                mapper_init = self.mapper_init,
                mapper = self.mapper,
                mapper_final = self.mapper_final,
                reducer = self.reducer
                )]
    
    def mapper_init(self):
        self.chordProgressions = {}
        #firstChords = ["a", "a5", "a6", "a7sus4", "aadd9", "ab", "ab5", "abm", "abm7", "absus4", "am", "am7", "am9", "amaj9", "ao", "asus", "asus2", "asus4", "b", "b7", "bb", "bb5", "bb6sus2", "bb7", "bbm", "bbm7", "bbsus2", "bbsus4", "bm", "bm7", "bsus4", "c", "c5", "c7", "c7m", "cadd9", "cm", "cmaj7", "csus2", "d", "d5", "d7", "dadd4", "db", "db5", "dm", "dm7", "dsus", "dsus2", "dsus4", "e", "e5", "e7", "eadd9", "eb", "eb5", "ebm", "ebm7", "ebsus2", "em", "em7", "emaj9", "f", "f5", "f7", "fm", "fm7", "fmaj7", "fsus2", "g", "g13", "g5", "g6", "g7", "g9", "gb", "gb5", "gbm", "gbm7", "gbm9", "gm", "gmaj7", "go", "gsus", "gsus4"]
        firstChords = ["A-5", "A-6", "A-7-Suspended-4", "A-Add-9", "A-Flat-Minor-7", "A-Flat-Suspended-4", "A-Diminished", "A-Suspended", "A-Suspended-4", "B-Flat-6-Suspended-2", "B-Flat-7", "B-Flat-Minor", "B-Flat-Minor-7", "B-Flat-Suspended-4", "B-Suspended-4", "C-7", "D-5", "D-Add-4", "D-Minor-7", "D-Suspended", "E-5", "E-Add-9", "E-Flat-Minor", "E-Flat-Suspended-2", "F-7", "G-13", "G-5", "G-7", "G-9", "G-Flat-Major", "G-Flat-5", "G-Flat-Minor-7", "G-Flat-Minor-9", "G-Diminished", "G-Suspended", "G-Suspended-4", "G-Major", "C-Major", "D-Major", "E-Minor", "A-Minor", "A-Major", "F-Major", "E-Major", "B-Minor",  "B-Major", "C-Add-9", "D-Minor", "B-Flat-Major", "A-Flat-Major", "D-Flat-5", "A-Suspended-2", "A-Minor-7", "E-Flat-Major", "B-Flat-Five", "A-Flat-5", "A-Flat-Minor", "B-Flat-Suspended-2", "F-Minor", "E-Minor-7", "F-Minor-7", "D-suspended-4", "D-7", "F-Suspended-2", "D-Flat-Major", "E-Major-9", "F-5", "E-Flat-5", "D-Suspended-2", "E-Flat-Minor-7", "C-Major-7", "G-Flat-Minor", "C-Suspended-2", "F-Major-7", "A-Minor-9", "B-7", "C-5", "C-Minor", "B-Minor-7", "C-Minor-7", "G-6", "A-Major-9", "E-7", "G-Minor", "G-Major-7"]       
        outputChordKeys = ["A-5", "A-6", "A-7-Suspended-4", "A-Add-9", "A-Flat-Minor-7", "A-Flat-Suspended-4", "A-Diminished", "A-Suspended", "A-Suspended-4", "B-Flat-6-Suspended-2", "B-Flat-7", "B-Flat-Minor", "B-Flat-Minor-7", "B-Flat-Suspended-4", "B-Suspended-4", "C-7", "D-5", "D-Add-4", "D-Minor-7", "D-Suspended", "E-5", "E-Add-9", "E-Flat-Minor", "E-Flat-Suspended-2", "F-7", "G-13", "G-5", "G-7", "G-9", "G-Flat-Major", "G-Flat-5", "G-Flat-Minor-7", "G-Flat-Minor-9", "G-Diminished", "G-Suspended", "G-Suspended-4", "G-Major", "C-Major", "D-Major", "E-Minor", "A-Minor", "A-Major", "F-Major", "E-Major", "B-Minor",  "B-Major", "C-Add-9", "D-Minor", "B-Flat-Major", "A-Flat-Major", "D-Flat-5", "A-Suspended-2", "A-Minor-7", "E-Flat-Major", "B-Flat-Five", "A-Flat-5", "A-Flat-Minor", "B-Flat-Suspended-2", "F-Minor", "E-Minor-7", "F-Minor-7", "D-suspended-4", "D-7", "F-Suspended-2", "D-Flat-Major", "E-Major-9", "F-5", "E-Flat-5", "D-Suspended-2", "E-Flat-Minor-7", "C-Major-7", "G-Flat-Minor", "C-Suspended-2", "F-Major-7", "A-Minor-9", "B-7", "C-5", "C-Minor", "B-Minor-7", "C-Minor-7", "G-6", "A-Major-9", "E-7", "G-Minor", "G-Major-7"]
        genres = ["all_Genres", "rock", "pop", "folk", "country"]
        songKeys= ["all_Keys", "a-flat-major", "a-major", "a-minor", "b-flat-major", "b-major", "b-minor", "c-major", "d-flat-major", "d-flat-minor", "d-major", "d-minor", "e-flat-major", "e-flat-minor", "e-major", "e-minor", "f-major", "f-minor", "f-sharp-major", "g-major", "g-minor"]
        # create master dictionary with fixed indexes
        self.chordProgressions = {}
        for firstChord in firstChords:
            self.chordProgressions[firstChord] = {}
            for songKey in songKeys:
                self.chordProgressions[firstChord][songKey] = {}
                for genre in genres:
                    self.chordProgressions[firstChord][songKey][genre] = {}
                    # set default values to 0 for eached chord in fixed index
                    for chord in outputChordKeys:
                        self.chordProgressions[firstChord][songKey][genre][chord] = 0
        self.chordConversion = {"a5": "A-5", "a6": "A-6", "a7sus4": "A-7-Suspended-4", "aadd9": "A-Add-9", "abm7": "A-Flat-Minor-7", "absus4": "A-Flat-Suspended-4", "ao": "A-Diminished", "asus": "A-Suspended", "asus4": "A-Suspended-4", "bb6sus2": "B-Flat-6-Suspended-2", "bb7": "B-Flat-7", "bbm": "B-Flat-Minor", "bbm7": "B-Flat-Minor-7", "bbsus4": "B-Flat-Suspended-4", "bsus4": "B-Suspended-4", "c7": "C-7", "d5": "D-5", "dadd4": "D-Add-4", "dm7": "D-Minor-7", "dsus": "D-Suspended", "e5": "E-5", "eadd9": "E-Add-9", "ebm": "E-Flat-Minor", "ebsus2": "E-Flat-Suspended-2", "f7": "F-7", "g13": "G-13", "g5": "G-5", "g7": "G-7", "g9": "G-9", "gb": "G-Flat-Major", "gb5": "G-Flat-5", "gbm7": "G-Flat-Minor-7", "gbm9": "G-Flat-Minor-9", "go": "G-Diminished", "gsus": "G-Suspended", "gsus4": "G-Suspended-4", "g": "G-Major", "c": "C-Major", "d": "D-Major", "em": "E-Minor", "am": "A-Minor", "a": "A-Major", "f": "F-Major", "e": "E-Major", "bm": "B-Minor",  "b": "B-Major", "cadd9": "C-Add-9", "dm": "D-Minor", "bb": "B-Flat-Major", "ab": "A-Flat-Major", "db5": "D-Flat-5", "asus2": "A-Suspended-2", "am7": "A-Minor-7", "eb": "E-Flat-Major", "bb5": "B-Flat-Five", "ab5": "A-Flat-5", "abm": "A-Flat-Minor", "bbsus2": "B-Flat-Suspended-2", "fm": "F-Minor", "em7": "E-Minor-7", "fm7": "F-Minor-7", "dsus4": "D-suspended-4", "d7": "D-7", "fsus2": "F-Suspended-2", "db": "D-Flat-Major", "emaj9": "E-Major-9", "f5": "F-5", "eb5": "E-Flat-5", "dsus2": "D-Suspended-2", "ebm7": "E-Flat-Minor-7", "cmaj7": "C-Major-7", "gbm": "G-Flat-Minor", "csus2": "C-Suspended-2", "fmaj7": "F-Major-7", "am9": "A-Minor-9", "b7": "B-7", "c5": "C-5", "cm": "C-Minor", "bm7": "B-Minor-7", "c7m": "C-Minor-7", "g6": "G-6", "amaj9": "A-Major-9", "e7": "E-7", "gm": "G-Minor", "gmaj7": "G-Major-7"}
        
    def mapper(self, _, line):
        line = line.strip()
        data = re.split(";",line.lower())
        song = data[0]
        genre = data[1]
        songKey = data[2].replace(" ","-")
        chordString = re.sub(r'[\[\]]','',data[3])
        chords = re.split(",",chordString)
        # iterate through all two-chord progressionss in order
        for i in range(0, len(chords)-1):
            # print self.chordProgressions
            # create a dictionary key that includes the "key" (tone) of the song
            firstChord = self.chordConversion[chords[i].replace("'","")]
            secondChord = self.chordConversion[chords[i+1].replace("'","")]
            self.chordProgressions[firstChord][songKey][genre][secondChord] += 1
            self.chordProgressions[firstChord][songKey]["all_Genres"][secondChord] += 1
            self.chordProgressions[firstChord]["all_Keys"]["all_Genres"][secondChord] += 1
            self.chordProgressions[firstChord]["all_Keys"][genre][secondChord] += 1
                
                
    def mapper_final(self):
        for key in self.chordProgressions.keys():
            # print key+";"+json.dumps(self.chordProgressions[key])
            yield key, json.dumps(self.chordProgressions[key])

    def reducer(self, key, value):
        firstChord = key
        self.chordProgressions = {}
        self.chordProgressions[firstChord] = {}
        for v in value:
            chord_dict = json.loads(v)
            for songKey in chord_dict.keys():
                if songKey not in self.chordProgressions[firstChord].keys():
                    # create sub-array for data within this songKey
                    self.chordProgressions[firstChord][songKey] = {}
                for genre in chord_dict[songKey].keys():
                    if genre not in self.chordProgressions[firstChord][songKey].keys():
                        self.chordProgressions[firstChord][songKey][genre] = {}
                    for secondChord in chord_dict[songKey][genre]:
                        if secondChord not in self.chordProgressions[firstChord][songKey][genre]:
                            self.chordProgressions[firstChord][songKey][genre][secondChord] = chord_dict[songKey][genre][secondChord]
                        else:
                            self.chordProgressions[firstChord][songKey][genre][secondChord] += chord_dict[songKey][genre][secondChord]
        yield firstChord, json.dumps(self.chordProgressions[firstChord]) 

        
if __name__ == '__main__':
    MRbuildStripes_detailed.run()

Overwriting buildStripes_detailed.py


In [346]:
!python buildStripes_detailed.py processed_songs_v1.txt > processed_song_list_detailed.txt

No configs found; falling back on auto-configuration
Creating temp directory /tmp/buildStripes_detailed.root.20181208.235251.733995
Running step 1 of 1...
Streaming final output from /tmp/buildStripes_detailed.root.20181208.235251.733995/output...
Removing temp directory /tmp/buildStripes_detailed.root.20181208.235251.733995...


In [327]:
%%writefile normalizedChordProgressions_detailed.py
#!~/anaconda2/bin/python
# -*- coding: utf-8 -*-

from __future__ import division
import collections
import re
import json
import math
import numpy as np
import itertools
import mrjob
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep
from collections import defaultdict
import collections as cl
import itertools

class MRnormalizedChordProgressions_detailed(MRJob):
    
  #START SUDENT CODE531_INV_INDEX
    SORT_VALUES = True
    def steps(self):
        JOBCONF_STEP = { 
          ######### IMPORTANT: THIS WILL HAVE NO EFFECT IN -r local MODE. MUST USE -r hadoop FOR SORTING #############
            'mapreduce.job.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapreduce.partition.keycomparator.options':'-k1',
        }
        return [
                MRStep(
                    jobconf=JOBCONF_STEP,
                    mapper_init = self.mapper_init,
                    mapper = self.mapper,
                    mapper_final = self.mapper_final,
                    reducer_init =self.reducer_init,
                    reducer = self.reducer,  
                    reducer_final = self.reducer_final)
                ]
    
    def mapper_init(self):
        self.chordProgressions = {}
    
    def mapper(self, _, line):
        key, value = line.strip().split('\t')
        self.chordProgressions[key] = {}
        self.chordProgressions[key] = json.loads(value)
        # iterate through the song keys 
        for songKey in self.chordProgressions[key].keys():
            # find the total count of chord progressions in the diectionary
            for genre in self.chordProgressions[key][songKey].keys():
                normalizationCount = 0
                for nextChord in self.chordProgressions[key][songKey][genre].keys():
                    normalizationCount += int(self.chordProgressions[key][songKey][genre][nextChord])
                for nextChord in self.chordProgressions[key][songKey][genre].keys():
                    if normalizationCount != 0:
                        self.chordProgressions[key][songKey][genre][nextChord] = int(self.chordProgressions[key][songKey][genre][nextChord])/normalizationCount  
                    else:
                        self.chordProgressions[key][songKey][genre][nextChord] = 0
    
    def mapper_final(self):
        for key in self.chordProgressions.keys():
            yield key , json.dumps(self.chordProgressions[key])
    
    
    def reducer_init(self):
        self.full_dict = {}
    
    # the reducer is purely being used for sortation
    def reducer(self, key, value):
        firstChord  = key.replace(r"'","")
        # print str('"'+firstChord+'", ')
        for v in value:
            temp = json.loads(v)
        self.full_dict[firstChord] = {}
        self.full_dict[firstChord] = temp
        #print key, json.dumps(temp), "\n"
        
    def reducer_final(self):
        # yield "chord_dict", json.dumps(self.full_dict)
        print json.dumps(self.full_dict), "\n"
        
if __name__ == '__main__':
    MRnormalizedChordProgressions_detailed.run()

Overwriting normalizedChordProgressions_detailed.py


In [347]:
!python normalizedChordProgressions_detailed.py processed_song_list_detailed.txt > normalized_song_list_detailed.txt

No configs found; falling back on auto-configuration
Creating temp directory /tmp/normalizedChordProgressions_detailed.root.20181208.235323.257532
Running step 1 of 1...
Streaming final output from /tmp/normalizedChordProgressions_detailed.root.20181208.235323.257532/output...
Removing temp directory /tmp/normalizedChordProgressions_detailed.root.20181208.235323.257532...
